In [ ]:
# Required packages

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

In [ ]:
class hh_Hamiltonian:
    "Energy preserved neural network"
    def __init__(self, solver, input_dim, output_dim, hidden_layer, each_neuron, activation_func, epoch,
                     batch_size, learning_rate, h_for_t_created):
        
        self.T, self.V, self.z, self.t, self.t_interval, self.initial_point, self.h, self.z_0, self.t_0 = solver

        # Neural Network Model with MSE Loss
        self.model = self.NN(inputs =input_dim, output=output_dim, hidden_layer=hidden_layer, neuron=each_neuron,
                                     activation=activation_func)
        print("model",self.model.weights)

        self.model_train = self.train(model=self.model, inputs=self.t, target=self.z, learning_rate=learning_rate,
                                    loss="mse", batch_size=batch_size, epochs=epoch)
        
        "*******************"
        t_created_0 = np.sort(np.random.uniform(float(self.t_interval[0]), float(self.t_interval[1]), h_for_t_created * 5))

        for i in range(0,100):
            print("******************************",i, "********************************************")
            t_created_1 = np.sort(np.random.choice(t_created_0, h_for_t_created, replace=False))
            target_created_1 = self.model_train(t_created_1)

            print("model_train",self.model_train.weights)
            t_created_2 = np.concatenate((t_created_1, self.t), axis=0)
            target_created_2 = np.concatenate((target_created_1, self.z), axis=0)

            self.energy_model_train = self.train(model=self.model, inputs=t_created_2, target=target_created_2, learning_rate=learning_rate,
                                        loss=self.custom_loss, batch_size=batch_size, epochs=10)
            print("energy_model_train", self.energy_model_train.weights)
        "*******************"
        #Neural Network Model with Energy Preserved Loss
        """
        self.energy_model = self.NN(inputs= input_dim,output=output_dim, hidden_layer=hidden_layer, neuron=each_neuron,
                                        activation=activation_func)

        # Parametreleri atama ve veri arttırma
        self.energy_model.set_weights(self.model_train.get_weights())
        t_created_0 = np.sort(np.random.uniform(float(self.t_interval[0]), float(self.t_interval[1]), h_for_t_created * 5))

        for i in range(0,5):

          t_created_1 = np.sort(np.random.choice(t_created_0, h_for_t_created, replace=False))
          target_created_1 = self.model_train(t_created_1)

          #adding solver points to the end of t_created_0
          t_created_2 = np.concatenate((t_created_1, self.t), axis=0)
          target_created_2 = np.concatenate((target_created_1, self.z), axis=0)
          print("********************************************",i,"************************************************")
          self.energy_model_train = self.train(model=self.energy_model, inputs=t_created_2, target=target_created_2,
                                              learning_rate=learning_rate, loss=self.custom_loss, batch_size=batch_size,
                                              epochs=500)
        """
    # Hamiltonian function
    def H(self, T, V, x, y, px, py):
        return T([px, py]) + V([x, y])

    # Neural network with MSE loss
    def NN(self, inputs, output, hidden_layer, neuron, activation="relu"):
        #np.random.seed(1)
        #tf.random.set_seed(1)
        
        inp = Input(shape=(inputs,), name= "Input_Layer")
        for i in range(0, hidden_layer):
            if i == 0:
                x = Dense(neuron, activation=activation, name="Hidden_Layer_{}".format(i+1))(inp)
            else:
                x = Dense(neuron, activation=activation, name="Hidden_Layer_{}".format(i+1))(x)
        x = Dense(output, name="Output_Layer")(x)
        """
         out = tf.constant([np.array(self.initial_point)],
                              dtype=tf.float32) + (1-tf.math.exp(-inp**2)) * x
        """
        out = tf.constant(np.array(self.initial_point),
                              dtype=tf.float32) + (1-tf.math.exp(-inp**2)) * x

        return Model(inputs=inp, outputs=out)

    # Neural Network Model Compile and Fit
    def train(self, model, inputs, target, learning_rate=0.0003, loss="mse", batch_size=32, epochs=1000):
        
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=opt, loss=loss)
        model.fit(x=inputs, y=target, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=True)
        
        return model
    """+  0.001 * tf.keras.losses.MSE(y_true,y_pred)"""
    def custom_loss(self, y_true, y_pred):
        return tf.keras.losses.MSE(self.H(self.T, self.V, self.initial_point[0],self.initial_point[1], self.initial_point[2], self.initial_point[3]),
                self.H(self.T, self.V, y_pred[None,:,0], y_pred[None, :,1], y_pred[None, :,2], y_pred[None, :,3])) +  0.001 * tf.keras.losses.MSE(y_true[None,-self.z.shape[0],:], y_pred[None,-self.z.shape[0],:])
